Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (18724, 28, 28), (18724,))


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 784), (200000, 10))
('Validation set', (10000, 784), (10000, 10))
('Test set', (18724, 784), (18724, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Introduce L2 Regularization for the Multinomial Logistic Regression model in tensorflow

In [5]:
batch_size = 128

# define the input variable
# the input variable will receive the image's pixels for every batch
tf_train_dataset = tf.placeholder(tf.float32, [None, image_size * image_size]) # batch_sizex784 matrix 
tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)

# define the weights' parameters
W = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels])) # 784x10 Matrix 

# define the biases
b = tf.Variable(tf.zeros([num_labels]))

logits = tf.matmul(tf_train_dataset, W) + b
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

regularizers = tf.nn.l2_loss(W)
loss += 5e-4 * regularizers 

# optimize the loss function using gradient descent
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

train_predictions = tf.nn.softmax(logits)

# run the validation dataset using the trained netword (weights and biases)
valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, W) + b)

# run the test dataset in the trained netword (weights and biases)
test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, W) + b)


Let's now train the model

- It is worth noting that the validation dataset is ran in each interation of the traing phase. So as the batches go and improve the parameters, the valid dataset is ran over the graph using these weights and biases recently improved.

- At the end, the test dataset in ran over the network to evaluate the network as a whole since the weights and biases have already finished training

In [6]:
# properly initialize the tensorflow variables
init = tf.initialize_all_variables()

# initialize the model in run the operation to initialize the variables
sess = tf.Session()
sess.run(init)

num_steps = 3001

for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    opt, l, predictions = sess.run(
      [optimizer, loss, train_predictions], feed_dict=feed_dict)
    
    if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy %.1f%%" % accuracy(valid_prediction.eval(session=sess), valid_labels))
        
print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(session=sess), test_labels))

Minibatch loss at step 0: 19.405302
Minibatch accuracy 5.5%
Validation accuracy 10.6%
Minibatch loss at step 500: 2.438282
Minibatch accuracy 73.4%
Validation accuracy 75.3%
Minibatch loss at step 1000: 1.817489
Minibatch accuracy 75.0%
Validation accuracy 77.1%
Minibatch loss at step 1500: 1.412781
Minibatch accuracy 79.7%
Validation accuracy 79.1%
Minibatch loss at step 2000: 1.452242
Minibatch accuracy 78.1%
Validation accuracy 79.2%
Minibatch loss at step 2500: 1.007788
Minibatch accuracy 82.8%
Validation accuracy 80.3%
Minibatch loss at step 3000: 1.046847
Minibatch accuracy 77.3%
Validation accuracy 80.1%
Test accuracy: 87.9%


Now, lets introduce the L2 regularization to a 2-layer neural network.

In [25]:
batch_size = 128

# define the input variable
# the input variable will receive the image's pixels for every batch
tf_train_dataset = tf.placeholder(tf.float32, [None, image_size * image_size]) # [128 x 784] matrix 
tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

# load the valid and test datasets
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)

# define the hidden layer size
hidden_layer_size = 1024

# define the weights (parameters) of the first layer
W_layer1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_layer_size])) # [784x1024] Matrix 

# define the biases for the firtst layer
b_layer1 = tf.Variable(tf.zeros([hidden_layer_size]))

# training computation
hidden_layer = tf.matmul(tf_train_dataset, W_layer1) + b_layer1 # [128 x 1024] Matrix

# apply the relu (Rectified Linear Regression) function 
relu_hidden_layer = tf.nn.relu(hidden_layer)

# define the parameters for the second layer
W_layer2 = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels])) # [1024 x 10] Matrix

# define the biases for the second layer 
b_layer2 = tf.Variable(tf.zeros([num_labels]))

# training computation
logits = tf.matmul(relu_hidden_layer, W_layer2) + b_layer2 # [128 x 10]

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

regularizers = tf.nn.l2_loss(W_layer1) + tf.nn.l2_loss(W_layer2)
loss += 5e-4 * regularizers 

# optimize the loss function using gradient descent
optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

train_predictions = tf.nn.softmax(logits)

# run the validation dataset using the trained netword (weights and biases)
valid_prediction_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, W_layer1) + b_layer1)
valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction_hidden_layer, W_layer2) + b_layer2)

# run the test dataset in the trained netword (weights and biases)
test_prediction_hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, W_layer1) + b_layer1)
test_prediction = tf.nn.softmax(tf.matmul(test_prediction_hidden_layer, W_layer2) + b_layer2)

Lets now train the network.

In [27]:
# properly initialize the tensorflow variables
init = tf.initialize_all_variables()

# initialize the model in run the operation to initialize the variables
sess = tf.Session()
sess.run(init)

num_steps = 3001

for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    opt, l, predictions = sess.run(
      [optimizer, loss, train_predictions], feed_dict=feed_dict)
    
    if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy %.1f%%" % accuracy(valid_prediction.eval(session=sess), valid_labels))
        print("Subset: %d - %d" % (offset, offset + batch_size))
        
print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(session=sess), test_labels))

Minibatch loss at step 0: 602.456726
Minibatch accuracy 9.4%
Validation accuracy 15.2%
Subset: 0 - 128
Minibatch loss at step 500: 175.481293
Minibatch accuracy 77.3%
Validation accuracy 78.7%
Subset: 64000 - 64128
Test accuracy: 85.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [54]:
batch_size = 128

# define the input variable
# the input variable will receive the image's pixels for every batch
tf_train_dataset = tf.placeholder(tf.float32, [None, image_size * image_size]) # [128 x 784] matrix 
tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

# load the valid and test datasets
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)

# define the hidden layer size
hidden_layer_size = 1024

# define the weights (parameters) of the first layer
W_layer1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_layer_size])) # [784x1024] Matrix 

# define the biases for the firtst layer
b_layer1 = tf.Variable(tf.zeros([hidden_layer_size]))

# training computation
hidden_layer = tf.matmul(tf_train_dataset, W_layer1) + b_layer1 # [128 x 1024] Matrix

# apply the relu (Rectified Linear Regression) function 
relu_hidden_layer = tf.nn.relu(hidden_layer)

# define the parameters for the second layer
W_layer2 = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels])) # [1024 x 10] Matrix

# define the biases for the second layer 
b_layer2 = tf.Variable(tf.zeros([num_labels]))

# training computation
logits = tf.matmul(relu_hidden_layer, W_layer2) + b_layer2 # [128 x 10]

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

# regularizers = tf.nn.l2_loss(W_layer1) + tf.nn.l2_loss(W_layer2)
# loss += 5e-4 * regularizers 

# optimize the loss function using gradient descent
optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

train_predictions = tf.nn.softmax(logits)

# run the validation dataset using the trained netword (weights and biases)
valid_prediction_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, W_layer1) + b_layer1)
valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction_hidden_layer, W_layer2) + b_layer2)

# run the test dataset in the trained netword (weights and biases)
test_prediction_hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, W_layer1) + b_layer1)
test_prediction = tf.nn.softmax(tf.matmul(test_prediction_hidden_layer, W_layer2) + b_layer2)

In [56]:
# properly initialize the tensorflow variables
init = tf.initialize_all_variables()

# initialize the model in run the operation to initialize the variables
sess = tf.Session()
sess.run(init)

num_steps = 101

for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    opt, l, predictions = sess.run(
      [optimizer, loss, train_predictions], feed_dict=feed_dict)
    
    if (step % 20 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy %.1f%%" % accuracy(valid_prediction.eval(session=sess), valid_labels))
        
print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(session=sess), test_labels))

Minibatch loss at step 0: 427.688782
Minibatch accuracy 10.2%
Validation accuracy 12.5%
Minibatch loss at step 20: 70.872391
Minibatch accuracy 65.6%
Validation accuracy 68.8%
Minibatch loss at step 40: 53.254601
Minibatch accuracy 69.5%
Validation accuracy 71.8%
Minibatch loss at step 60: 56.218655
Minibatch accuracy 71.1%
Validation accuracy 73.4%
Minibatch loss at step 80: 40.858280
Minibatch accuracy 76.6%
Validation accuracy 75.2%
Minibatch loss at step 100: 55.675922
Minibatch accuracy 71.9%
Validation accuracy 71.8%
Test accuracy: 78.0%


Doing only 101 (from 0 to 100) batches, each one of size 128, the final test accuracy was 82.3% (Without Regularization). If this approach was to overfit the model, I was expecting that the Minibaches accuracy went up. However, at step 100, the Minibatch accuracy was 74.2%.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
